# Data Cleaning

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter

In [23]:
df = pd.read_csv("../data/iot_telemetry_data.csv")
df.head()

,ts,device,co,humidity,light,lpg,motion,smoke,temp
0,1.594512e+09,b8:27:eb:bf:9d:51,0.004956,51.000000,False,0.007651,False,0.020411,22.700000
1,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.700001
2,1.594512e+09,b8:27:eb:bf:9d:51,0.004976,50.900000,False,0.007673,False,0.020475,22.600000
3,1.594512e+09,1c:bf:ce:15:ec:4d,0.004403,76.800003,True,0.007023,False,0.018628,27.000000
4,1.594512e+09,b8:27:eb:bf:9d:51,0.004967,50.900000,False,0.007664,False,0.020448,22.600000


In [24]:
df.describe()

,ts,co,humidity,lpg,smoke,temp
count,4.051840e+05,405184.000000,405184.000000,405184.000000,405184.000000,405184.000000
mean,1.594858e+09,0.004639,60.511694,0.007237,0.019264,22.453987
std,1.994984e+05,0.001250,11.366489,0.001444,0.004086,2.698347
min,1.594512e+09,0.001171,1.100000,0.002693,0.006692,0.000000
25%,1.594686e+09,0.003919,51.000000,0.006456,0.017024,19.900000
50%,1.594858e+09,0.004812,54.900000,0.007489,0.019950,22.200000
75%,1.595031e+09,0.005409,74.300003,0.008150,0.021838,23.600000
max,1.595203e+09,0.014420,99.900002,0.016567,0.046590,30.600000


In [28]:
# Convert Timestamp to datetime
df['ts'] = pd.to_datetime(df['ts'], unit='s')

In [30]:
print(df.isnull().sum())
# Fill missing values with 0

ts          0
device      0
co          0
humidity    0
light       0
lpg         0
motion      0
smoke       0
temp        0
dtype: int64


In [31]:
df['temp'] = df['temp'].fillna(df['temp'].mean())

In [33]:
print(df.duplicated().sum())
# Remove duplicates

13


In [34]:
df = df.drop_duplicates()

In [38]:
df = df[(df['temp'] >= -30) & (df['temp'] <= 60)]
df = df[(df['humidity'] >= 0) & (df['humidity'] <= 100)]
df = df[(df['co'] >= 0) & (df['lpg'] >= 0) & (df['smoke'] >= 0)]


In [ ]:
from sqlalchemy import create_engine
user = "postgres"
password = "your_correct_password"
host = "localhost"
port = "5432"
database = "sensor_data"

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')
df.to_sql('iot_data', engine, index=False, if_exists='replace')


In [39]:

df['light'] = df['light'].astype(bool)
df['motion'] = df['motion'].astype(bool)

# Feature Engineering
df['temp_celsius'] = df['temp'] - 273.15
df['temp_fahrenheit'] = df['temp'] * (9/5) + 32
df['temp_kelvin'] = df['temp']

# Save cleaned data
df.to_csv("../data/cleaned_data.csv", index=False)

# Load cleaned data
df = pd.read_csv("../data/cleaned_data.csv")